<a href="https://colab.research.google.com/github/minjaekim070/Bootcamp_Project/blob/main/CNN_project_2%EC%9D%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

전체 데이터셋의 다운로드가 오래 걸려서 bottle 폴더에 대한 내용을 우선적으로 정상/비정상으로 분류할 수 있도록 진행

#Library import


In [2]:
# 패키지 import
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf
from torchvision import transforms, datasets, utils
from torchsummary import summary

import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.preprocessing import image


#이미지 업로드 및 배열 변환


In [3]:
# 이미지 경로와 카테고리 설정
data_dir = '/content/drive/MyDrive/bottle/train/good'

# 이미지 로딩 및 배열 변환
train_image_data = []
for img_name in os.listdir(data_dir):
    img_path = os.path.join(data_dir, img_name)
    img = image.load_img(img_path, target_size=(32, 32))  # 이미지를 지정된 크기로 로드합니다
    img_array = image.img_to_array(img)  # 이미지를 배열 형태로 변환합니다
    train_image_data.append(img_array)


In [4]:
# 배열로 변환
train_image_data = np.array(train_image_data)
train_image_data = train_image_data / 255

In [5]:
train_image_data.shape

(209, 32, 32, 3)

In [6]:
large_data_dir = '/content/drive/MyDrive/bottle/test/broken_large'
small_data_dir = '/content/drive/MyDrive/bottle/test/broken_small'
contamination_data_dir='/content/drive/MyDrive/bottle/test/contamination'
test_data_dir = '/content/drive/MyDrive/bottle/test/good'

# 이미지 로딩 및 전처리를 위한 ImageDataGenerator 생성
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)  # 이미지를 0과 1 사이의 값으로 정규화합니다

# test 이미지 로딩 및 배열 변환
test_image_data = []
for img_name in os.listdir(test_data_dir):
    img_path = os.path.join(test_data_dir, img_name)
    img = image.load_img(img_path, target_size=(32, 32))  # 이미지를 지정된 크기로 로드합니다
    img_array = image.img_to_array(img)  # 이미지를 배열 형태로 변환합니다
    test_image_data.append(img_array)

large_image_data = []
for img_name in os.listdir(large_data_dir):
    img_path = os.path.join(large_data_dir, img_name)
    img = image.load_img(img_path, target_size=(32, 32))  # 이미지를 지정된 크기로 로드합니다
    img_array = image.img_to_array(img)  # 이미지를 배열 형태로 변환합니다
    large_image_data.append(img_array)

small_image_data = []
for img_name in os.listdir(small_data_dir):
    img_path = os.path.join(small_data_dir, img_name)
    img = image.load_img(img_path, target_size=(32, 32))  # 이미지를 지정된 크기로 로드합니다
    img_array = image.img_to_array(img)  # 이미지를 배열 형태로 변환합니다
    small_image_data.append(img_array)

contamination_image_data = []
for img_name in os.listdir(contamination_data_dir):
    img_path = os.path.join(contamination_data_dir, img_name)
    img = image.load_img(img_path, target_size=(32, 32))  # 이미지를 지정된 크기로 로드합니다
    img_array = image.img_to_array(img)  # 이미지를 배열 형태로 변환합니다
    contamination_image_data.append(img_array)

In [7]:
# 배열로 변환
test_image_data = np.array(test_image_data)
small_image_data = np.array(small_image_data)
large_image_data = np.array(large_image_data)
contamination_image_data = np.array(contamination_image_data)

test_image_data = test_image_data / 255
small_image_data = small_image_data / 255
large_image_data = large_image_data / 255
contamination_image_data = contamination_image_data / 255

In [8]:
test_image_data[0][16][16]

array([0.1882353 , 0.16470589, 0.2       ], dtype=float32)

#모델 구성 및 결과 예측

In [ ]:
#준비된 데이터 셋
"""
train_image_data : 정상
test_image_data : 정상
small_image_data : 파손
large_image_data : 파손
contamination_image_data : 오염
"""

In [36]:
# 이미지 데이터 통합
all_images = np.concatenate((test_image_data, large_image_data, contamination_image_data), axis=0)

# 이미지 리사이즈
resized_images = []
for image in all_images:
    resized_image = tf.image.resize(image, (32, 32))
    resized_images.append(resized_image)

# 이미지 데이터 변환
resized_images = np.array(resized_images)
resized_images = resized_images.astype(np.float32)

# 레이블 생성
train_labels = np.array([[1, 0, 0]] * len(test_image_data))  # 정상: [1, 0, 0]
broken_labels = np.array([[0, 1, 0]] * len(large_image_data))  # 파손: [0, 1, 0]
contamination_labels = np.array([[0, 0, 1]] * len(contamination_image_data))  # 오염: [0, 0, 1]

all_labels = np.concatenate((train_labels, broken_labels, contamination_labels), axis=0)


In [37]:
print(resized_images.shape)
print(all_labels.shape)

(209, 32, 32, 3)
(61, 32, 32, 3)
(61, 3)


In [29]:
import tensorflow as tf
from tensorflow.keras import layers, models

train_labels = [[1, 0, 0],   # 첫 번째 이미지의 레이블: [정상, 파손, 오염] = [1, 0, 0]
                [0, 1, 0],   # 두 번째 이미지의 레이블: [정상, 파손, 오염] = [0, 1, 0]
                [0, 0, 1]]  # 세 번째 이미지의 레이블: [정상, 파손, 오염] = [0, 0, 1]
# CNN 모델 구성
model = models.Sequential()

#Layer 1
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))

#Layer 2
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

#1D
model.add(layers.Flatten())

#Dense
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# 모델 요약 출력
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 2304)              0         
                                                                 
 dense_4 (Dense)             (None, 64)               

In [38]:
# 모델 학습
model.fit(resized_images, all_labels, epochs=10, batch_size=32)

Epoch 1/10
2/2 [==============================] - 0s 153ms/step - loss: 1.2209 - accuracy: 0.3279
Epoch 2/10
2/2 [==============================] - 0s 8ms/step - loss: 1.0523 - accuracy: 0.4098
Epoch 3/10
2/2 [==============================] - 0s 8ms/step - loss: 1.0757 - accuracy: 0.3279
Epoch 4/10
2/2 [==============================] - 0s 10ms/step - loss: 1.0570 - accuracy: 0.3607
Epoch 5/10
2/2 [==============================] - 0s 8ms/step - loss: 1.0077 - accuracy: 0.7705
Epoch 6/10
2/2 [==============================] - 0s 8ms/step - loss: 1.0156 - accuracy: 0.5738
Epoch 7/10
2/2 [==============================] - 0s 7ms/step - loss: 1.0198 - accuracy: 0.4426
Epoch 8/10
2/2 [==============================] - 0s 8ms/step - loss: 1.0009 - accuracy: 0.4918
Epoch 9/10
2/2 [==============================] - 0s 8ms/step - loss: 0.9748 - accuracy: 0.7377
Epoch 10/10
2/2 [==============================] - 0s 8ms/step - loss: 0.9834 - accuracy: 0.7049


In [65]:
# 예측 수행
predictions = model.predict(small_image_data)

1/1 [==============================] - 0s 23ms/step


In [66]:
predictions

array([[0.22459038, 0.41832125, 0.35708836],
       [0.26837778, 0.35394242, 0.3776798 ],
       [0.28076318, 0.34335044, 0.37588638],
       [0.21695596, 0.41729066, 0.3657534 ],
       [0.23953864, 0.39052168, 0.36993966],
       [0.23197049, 0.41807228, 0.3499572 ],
       [0.2525344 , 0.36883736, 0.37862825],
       [0.26174194, 0.37689683, 0.3613613 ],
       [0.21095107, 0.4410752 , 0.3479738 ],
       [0.21937765, 0.43392098, 0.34670138],
       [0.16738124, 0.48860455, 0.3440142 ],
       [0.2857293 , 0.34534883, 0.36892188],
       [0.2671682 , 0.3757918 , 0.35703996],
       [0.2522643 , 0.3812298 , 0.36650592],
       [0.2617141 , 0.37796158, 0.36032426],
       [0.22715205, 0.41302842, 0.35981953],
       [0.25180107, 0.3920544 , 0.3561445 ],
       [0.17522094, 0.44503316, 0.37974584],
       [0.22348033, 0.41883025, 0.3576894 ],
       [0.26675838, 0.3613306 , 0.37191102],
       [0.25078943, 0.39182073, 0.35738984],
       [0.26858062, 0.35773948, 0.37367988]], dtype=flo

In [67]:
# 최종 예측 레이블 생성
predicted_labels = []
for prediction in predictions:
    if prediction[0] > prediction[1] and prediction[0] > prediction[2]:
        predicted_labels.append([1, 0, 0])
    elif prediction[1] > prediction[0] and prediction[1] > prediction[2]:
        predicted_labels.append([0, 1, 0])
    else:
        predicted_labels.append([0, 0, 1])
predicted_labels = np.array(predicted_labels)

In [69]:
count = 0
for label in predicted_labels:
    if np.array_equal(label, [0, 1, 0]):
        count += 1

prediction_accuracy = count / len(small_image_data) * 100
print('예측률: {}%'.format(prediction_accuracy))

예측률: 72.72727272727273%


broken_small에 대해서 어느정도는 잘 예측하는 모습

In [71]:
predictions = model.predict(train_image_data)
predicted_labels = []
for prediction in predictions:
    if prediction[0] > prediction[1] and prediction[0] > prediction[2]:
        predicted_labels.append([1, 0, 0])
    elif prediction[1] > prediction[0] and prediction[1] > prediction[2]:
        predicted_labels.append([0, 1, 0])
    else:
        predicted_labels.append([0, 0, 1])
predicted_labels = np.array(predicted_labels)

count = 0
for label in predicted_labels:
    if np.array_equal(label, [1, 0, 0]):
        count += 1

prediction_accuracy = count / len(train_image_data) * 100
print('예측률: {}%'.format(prediction_accuracy))


7/7 [==============================] - 0s 2ms/step
예측률: 0.0%


막상 정상 데이터에 대해서는 정확도가 0으로 나오는 모습